## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Conditions
0,0,Sobolevo,54.4299,31.8996,66.22,73,100,0.31,RU,overcast clouds
1,1,Ostrovnoy,68.0531,39.5131,49.12,83,15,8.61,RU,few clouds
2,2,Alihe,50.5667,123.7167,47.48,97,8,2.48,CN,clear sky
3,3,Kargasok,59.0556,80.8572,51.71,87,24,5.44,RU,few clouds
4,4,Richards Bay,-28.7830,32.0377,63.81,76,4,4.00,ZA,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Conditions
0,0,Sobolevo,54.4299,31.8996,66.22,73,100,0.31,RU,overcast clouds
4,4,Richards Bay,-28.7830,32.0377,63.81,76,4,4.00,ZA,clear sky
6,6,Saint-Philippe,-21.3585,55.7679,69.35,75,43,11.83,RE,scattered clouds
7,7,Kamenka,51.3223,42.7678,76.77,56,67,4.45,RU,broken clouds
8,8,Tura,25.5198,90.2201,73.60,98,87,3.44,IN,overcast clouds
10,10,Opuwo,-18.0607,13.8400,65.19,24,0,7.02,NaN,clear sky
12,12,Korla,41.7597,86.1469,77.81,16,100,4.61,CN,overcast clouds
14,14,Shiyan,32.6475,110.7781,78.87,84,1,2.95,CN,heavy intensity rain
18,18,Portland,45.5234,-122.6762,87.94,51,1,0.00,US,clear sky
20,20,Nacala,-14.5428,40.6728,70.81,74,4,8.95,MZ,clear sky


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       527
City          527
Lat           527
Lng           527
Max Temp      527
Humidity      527
Cloudiness    527
Wind Speed    527
Country       520
Conditions    527
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       520
City          520
Lat           520
Lng           520
Max Temp      520
Humidity      520
Cloudiness    520
Wind Speed    520
Country       520
Conditions    520
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Conditions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(16)

,City,Country,Max Temp,Conditions,Lat,Lng,Hotel Name
0,Sobolevo,RU,66.22,overcast clouds,54.4299,31.8996,
4,Richards Bay,ZA,63.81,clear sky,-28.7830,32.0377,
6,Saint-Philippe,RE,69.35,scattered clouds,-21.3585,55.7679,
7,Kamenka,RU,76.77,broken clouds,51.3223,42.7678,
8,Tura,IN,73.60,overcast clouds,25.5198,90.2201,
12,Korla,CN,77.81,overcast clouds,41.7597,86.1469,
14,Shiyan,CN,78.87,heavy intensity rain,32.6475,110.7781,
18,Portland,US,87.94,clear sky,45.5234,-122.6762,
20,Nacala,MZ,70.81,clear sky,-14.5428,40.6728,
21,Vao,NC,66.13,broken clouds,-22.6667,167.4833,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.count()

City          467
Country       467
Max Temp      467
Conditions    467
Lat           467
Lng           467
Hotel Name    467
dtype: int64

In [26]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Conditions}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))